# Data from Transfermarkt

In [1]:
#https://www.transfermarkt.com/lionel-messi/torenachminute/spieler/28003/saison//verein/0/liga/0/wettbewerb//pos/0/trainer_id/0/plus/1
import pandas as pd
import requests # http requests
import re # regular expressions
from bs4 import BeautifulSoup # xml parsing



In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def get_data_from_div_detailed(url):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.81 Safari/537.36'
    }

    response = requests.get(url, headers=headers)
    data = []
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Find the div with class 'responsive-table'
        div = soup.find('div', {'class': 'responsive-table'})
        
        if div:
            # Extract headers
            headers = div.find_all('th')
            header_texts = [header.get_text(strip=True) for header in headers]
            header_texts.append('Club')  # Adding 'Club' as a header
            
            # Extract contents
            rows = div.find_all('tr')
            for row in rows[1:]:  # Skipping the first row as it's the header row
                player_data = {}
                
                cells = row.find_all('td')
                cell_texts = [cell.get_text(strip=True) for cell in cells]
                
                # Checking if the row is complete (same length as headers)
                if len(cell_texts) == len(header_texts) - 1:
                    for index, header in enumerate(header_texts[:-1]):
                        player_data[header] = cell_texts[index]
                    
                    # Extracting the club name from the 'title' attribute
                    club_tag = cells[3].find('a')
                    if club_tag and 'title' in club_tag.attrs:
                        club_name = club_tag['title']
                        player_data['Club'] = club_name
                    
                    data.append(player_data)
                
        else:
            print('Div not found.')
    else:
        print('Failed to fetch data.')
    
    return data

messi = "https://www.transfermarkt.com/lionel-messi/torenachminute/spieler/28003/saison//verein/0/liga/0/wettbewerb//pos/0/trainer_id/0/plus/1"
ronaldo = "https://www.transfermarkt.com/cristiano-ronaldo/torenachminute/spieler/8198/saison//verein/0/liga/0/wettbewerb//pos/0/trainer_id/0/plus/1"
detailed_messi = get_data_from_div_detailed(messi)

detailed_ronaldo = get_data_from_div_detailed(ronaldo)

# Convert to DataFrame
df_messi_detailed = pd.DataFrame(detailed_messi)
df_ronaldo_detailed = pd.DataFrame(detailed_ronaldo)

print(df_messi_detailed)
#print(df_ronaldo)

   Season Competition reihenfolge ASC, saison_id DESC, verein_id ASC  \
0             Total :                                                  
1    2023                                                        MLS   
2    2023                                                Leagues Cup   
3   22/23                                                    Ligue 1   
4   22/23                                           Champions League   
..    ...         ...                                            ...   
65  05/06                                                     LaLiga   
66  05/06                                               Copa del Rey   
67  05/06                                           Champions League   
68  04/05                                                     LaLiga   
69  04/05                                           2ª B - Grupo III   

                   Club 1 - 15 16 - 30 31 - 45 45+ 46 - 60 61 - 75 76 - 90  \
0                           72     120     123   6     10

In [3]:
def get_data_from_div_trophies(url):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36'
    }
    response = requests.get(url, headers=headers)
    data = []
    
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Find all divs with class 'large-6 columns'
        divs = soup.find_all('div', {'class': 'large-6 columns'})
        
        for div in divs:
            # Extract headers within each 'large-6 columns' div
            headers = div.find_all('h2')
            
            for head in headers: 
                trophies_kind = re.search(r'>\s*([0-9]+x.*?)\s*<', str(head)) 
                if trophies_kind:
                    actual_trophies = trophies_kind.group(1)
                    
                    # Extract contents within each 'large-6 columns' div
                    rows = div.find_all('tr')
                    pat_2 = '[0-9]+'
                    
                    type_of_trophie_list = []
                    for row in rows[0:]:  # Skipping the first row as it's the header row
                        specific_trophie = re.search(pat_2, str(row)) 
                        if specific_trophie:
                            year = specific_trophie.group()
                            type_of_trophie_list.append(year)
                    
                    data.append((actual_trophies, type_of_trophie_list))
            
    else:
        print('Failed to fetch data.')
    
    return data

In [10]:
#messi = "https://www.transfermarkt.com/lionel-messi/torenachminute/spieler/28003/saison//verein/0/liga/0/wettbewerb//pos/0/trainer_id/0/plus/1"
#result_messi = get_data_from_div_(messi)
#df_messi_detailed = pd.DataFrame(result_messi)
#print(df_messi_detailed)

messi = "https://www.transfermarkt.com/mario-mandzukic/erfolge/spieler/28003"
trophies_messi = get_data_from_div_trophies(messi)
df_trophies_messi_trophies = pd.DataFrame(trophies_messi)

#print(final_years)




In [11]:

def correct_the_seasons(df):
    final_years = []
    for years_lists in df[1].values:
        four_digits = []
        for year in years_lists:
            if len(year) == 4: 
                four_digits.append(year)
            else:
                #print(year)
                incremented_year = '{:02d}'.format(int(year) + 1)

                seasons = year+'/'+incremented_year
                four_digits.append(seasons)
        final_years.append(four_digits)
    
    return final_years
        
correted_seasons = correct_the_seasons(df_trophies_messi_trophies)
#print(correted_seasons)
df_trophies_messi_trophies[2] = correted_seasons
#print(df_trophies_messi_trophies)

In [185]:
def get_data_from_div_national(url):
    import requests
    from bs4 import BeautifulSoup
    
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36'
    }
    response = requests.get(url, headers=headers)
    
    titles = []  # List to store all found titles
    
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Find all divs with class 'responsive-table'
        divs = soup.find_all('div', {'class': 'responsive-table'})
        
        # Skip the first div with the class 'responsive-table'
        for div in divs[1:]:
            if div:
                #print(div)
                span_elements = div.find_all('span', class_='icons_sprite')  # Using find_all to collect all span elements
                zentriert_elements = div.find_all('th', class_='zentriert')  # Using find_all to collect all z elements
                for element in zentriert_elements:
                    text = element.get_text(strip=True) 
                    titles.append(text)
                    #print(zentriert_elements)
                #Venue = div.find_all( class_='Venue')  # Using find_all to collect all span elements
                #print(Venue)
                
                count = 0
                for span_element in span_elements:
                    count+=1
                    title_text = span_element.get('title')
                    #print(count, title_text)
                    titles.append(title_text)  # Append each found title to the list
       
    
        for div in divs[1:]:
            if div:
                #print(div)
                td_elements = soup.select('tbody td.zentriert.hauptlink.no-border-rechts')
                #print(td_elements)
                # Extract the title from <a> tags within <td> elements
                special_lines = []
                for td in td_elements:
                    #print(td)
                    a_tag = td.find('a')
                    #print(a_tag)
                    if a_tag:
                        
                        reg = 'title="?[^"]+"?\/>'
                        special_line_1 = re.findall(reg, str(a_tag))
                        #print(special_line) 
                        
                        new_reg = '[^"]+'
                        special_line_2 = re.findall(new_reg, "".join(special_line_1))
                        #print(special_line_2[1])
                        
                        #special_line = a_tag.get('title')
                        #print(special_line_2)
                        special_lines.append(special_line_2[1])
                    
                desired_rows = div.find_all('tr')
                
                
                matches_list = []
                
                for div in desired_rows:
                    new_desired_row = div.find_all('td')
                    
                    match_details = []

                    for cell in new_desired_row:
                        text = cell.get_text(strip=True)
                        match_details.append(text if text else '0')

                    #matches_list.append(match_details)
                    #print(match_details, len(match_details))
                    
                    
                         #print(match_details[1]+match_details[3:4]+match_details[6])
                    if len(match_details) == 0: 
                        pass
                        
                    if len(match_details) == 2: 
                        type_of_matches = match_details+[0]*14
                        matches_list.append(type_of_matches)
                        
                    if len(match_details) == 11: 
                        not_played = [match_details[1]]+[match_details[3]]+[match_details[4]]+[match_details[6]]+match_details[8:11]+[0]*9
                        matches_list.append(not_played)
                    
                    if len(match_details) == 20: 
                        combined = [match_details[1]]+[match_details[3]]+[match_details[4]]+[match_details[6]]+ match_details[8:20]
                        matches_list.append(combined)
                    
                    

      
                
        else:
            print('Failed to fetch data.')
    
    #print(titles[1:5], titles[14:23])
    
    return matches_list

messi_national = "https://www.transfermarkt.com/lionel-messi/nationalmannschaft/spieler/28003/verein_id/3437/hauptwettbewerb//wettbewerb_id//start/2005-08-17/ende/2023-12-15/nurEinsatz/0/plus/1"
national_messi = get_data_from_div_national(messi_national)
National_column_names = ["Group", "Venue", "Date", "Home Team", "Away Team", "Result", "Pos", "Goal", "Assist", "Own Goal", "Yellow",
     "Double Yellow", "Red", "Substitutions on", "Substitutions off", "Minutes Played"]

# Create the pandas DataFrame 
national_messi_df = pd.DataFrame(national_messi, columns = National_column_names) 
 
#print(national_messi)  # This will print all the extracted titles as a list
#df_national_messi = pd.DataFrame(national_messi)

#print(df_national_messi)

Failed to fetch data.


In [180]:
for i in national_messi:
    print(len(i))

16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16


In [182]:
column_names = ["Group", "Venue", "Date", "Home Team", "Away Team", "Result", "Pos", "Goal", "Assist", "Own Goal", "Yellow",
     "Double Yellow", "Red", "Substitutions on", "Substitutions off", "Minutes Played"]

# Create the pandas DataFrame 
df = pd.DataFrame(national_messi, columns = column_names) 
 
    
    
    
    
# Permanently changes the pandas settings
#pd.set_option('display.max_rows', None)
#pd.set_option('display.max_columns', None)
#pd.set_option('display.width', None)
#pd.set_option('display.max_colwidth', -1)
 
# All dataframes hereafter reflect these changes.
display(df)
 
#print('**RESET_OPTIONS**')
 
# Resets the options
#pd.reset_option('all')
#display(df)



,Group,Venue,Date,Home Team,Away Team,Result,Pos,Goal,Assist,Own Goal,Yellow,Double Yellow,Red,Substitutions on,Substitutions off,Minutes Played
0,0,International Friendlies,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,Budapest,"Aug 17, 2005",Hungary,Argentina,1:2,SS,0,0,0,0,0,65',64',0,1'
2,0,World Cup qualification South America,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Group A,Asunción,"Sep 3, 2005",Paraguay,Argentina,1:0,SS,0,0,0,0,0,0,80',0,10'
4,Group A,Buenos Aires,"Oct 9, 2005",Argentina,Peru,2:0,SS,0,1,0,0,0,0,0,0,90'
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
297,Group A,La Paz,"Sep 12, 2023",Bolivia,Argentina,0:3,Muscle injury,0,0,0,0,0,0,0,0,0
298,Group A,Buenos Aires,"Oct 13, 2023",Argentina,Paraguay,1:0,RW,0,0,0,0,0,0,53',0,37'
299,Group A,Lima,"Oct 18, 2023",Peru,Argentina,0:2,RW,2,0,0,0,0,0,0,0,90'
300,Group A,"La Boca, Buenos Aires, Capital Federal","Nov 17, 2023",Argentina,Uruguay,0:2,RW,0,0,0,0,0,0,0,0,90'


**RESET_OPTIONS**


C:\Users\User\anaconda3\lib\site-packages\pandas\_config\config.py:653: FutureWarning: As the xlwt package is no longer maintained, the xlwt engine will be removed in a future version of pandas. This is the only engine in pandas that supports writing in the xls format. Install openpyxl and write to an xlsx file instead.
  warnings.warn(d.msg, FutureWarning)
C:\Users\User\anaconda3\lib\site-packages\pandas\_config\config.py:653: FutureWarning: 
: boolean
    use_inf_as_null had been deprecated and will be removed in a future
    version. Use `use_inf_as_na` instead.

  warnings.warn(d.msg, FutureWarning)


In [15]:
ls

 Volume in drive C is Windows-SSD
 Volume Serial Number is C4CE-7D5D

 Directory of C:\Users\User\OneDrive\Dokumenter\Dokumenter\MsC_ITU\Advanced_Applied_Statistics

12/15/2023  09:14 PM    <DIR>          .
11/08/2023  02:09 PM    <DIR>          ..
11/17/2023  02:37 PM    <DIR>          .ipynb_checkpoints
12/06/2023  01:05 PM           149,522 BallonDOr.csv
12/08/2023  10:40 AM           178,001 BallonDOr_combined.csv
09/14/2023  02:36 PM         9,163,272 casi_pdf.pdf
10/01/2023  08:11 PM    <DIR>          Exercise_03
11/16/2023  11:33 AM    <DIR>          Exercise_04
11/16/2023  10:40 AM    <DIR>          Exercise_05
10/17/2023  03:26 PM    <DIR>          Exercise_06
10/17/2023  01:27 PM    <DIR>          Exercise_07
11/17/2023  01:13 PM    <DIR>          Exercise_08
11/17/2023  01:22 PM    <DIR>          Exercise_09
11/12/2023  03:30 PM    <DIR>          Exercise_10
09/11/2023  08:27 AM    <DIR>          Exercise1
12/15/2023  09:14 PM           716,617 Get Data for Project.ipynb
09/

In [16]:
excel_file = pd.read_csv(r"BallonDOr_combined.csv")
print(excel_file)

      Year  Rank              Player                 Club   Nationality  \
0     1956     1    Stanley Matthews            Blackpool       England   
1     1956     2  Alfredo Di Stéfano       Real Madrid CF         Spain   
2     1956     3        Raymond Kopa       Real Madrid CF        France   
3     1956     4       Ferenc Puskás               Honvéd       Hungary   
4     1956     5          Lev Yashin        Dynamo Moskva  Soviet Union   
...    ...   ...                 ...                  ...           ...   
1895  2023    26       Jamal Musiala        Bayern Munich       Germany   
1896  2023    27      Nicolò Barella          Inter Milan         Italy   
1897  2023    28   Randal Kolo Muani  Eintracht Frankfurt        France   
1898  2023    28     Martin Ødegaard              Arsenal        Norway   
1899  2023    30          Rúben Dias      Manchester City      Portugal   

      Points   P1   P2   P3   P4   P5  Votes  RankPts   Share  Percent   Voted  
0       47.0  6.0 

In [17]:
#ballon_dor_df = pd.read_csv(r"BallonDOr.csv")
#print(ballon_dor_df["Year"]==2013)
excel_file = pd.read_csv(r"BallonDOr_combined.csv")
excel_file.loc[excel_file["Year"]>=2008]

,Year,Rank,Player,Club,Nationality,Points,P1,P2,P3,P4,P5,Votes,RankPts,Share,Percent,Voted
1498,2008,1,Cristiano Ronaldo,Manchester United,Portugal,446.0,77.0,11.0,4.0,1.0,3.0,96.0,50.0,0.3097,0.9292,1.0000
1499,2008,2,Lionel Messi,FC Barcelona,Argentina,281.0,6.0,33.0,27.0,14.0,10.0,90.0,25.0,0.1951,0.5854,0.9375
1500,2008,3,Fernando Torres,Liverpool,Spain,179.0,5.0,13.0,24.0,9.0,12.0,63.0,16.7,0.1243,0.3729,0.6563
1501,2008,4,Iker Casillas,Real Madrid CF,Spain,133.0,2.0,16.0,12.0,8.0,7.0,45.0,12.5,0.0924,0.2771,0.4688
1502,2008,5,Xavi,FC Barcelona,Spain,97.0,3.0,9.0,4.0,15.0,4.0,35.0,10.0,0.0674,0.2021,0.3646
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1895,2023,26,Jamal Musiala,Bayern Munich,Germany,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0000,NaN,NaN
1896,2023,27,Nicolò Barella,Inter Milan,Italy,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0000,NaN,NaN
1897,2023,28,Randal Kolo Muani,Eintracht Frankfurt,France,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0000,NaN,NaN
1898,2023,28,Martin Ødegaard,Arsenal,Norway,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0000,NaN,NaN


In [18]:
ronaldo = "https://www.transfermarkt.com/cristiano-ronaldo/torenachminute/spieler/8198/saison//verein/0/liga/0/wettbewerb//pos/0/trainer_id/0/plus/1"




In [19]:
#print(ballon_dor_df["Year"])
ballon_2008_2018 = excel_file.loc[excel_file["Year"]>=2008]

p_08_18 = ballon_2008_2018["Player"].unique()
print(p_08_18)

['Cristiano Ronaldo' 'Lionel Messi' 'Fernando Torres' 'Iker Casillas'
 'Xavi' 'Andrei Arshavin' 'David Villa' 'Kaká' 'Zlatan Ibrahimović'
 'Steven Gerrard' 'Marcos Senna' 'Emmanuel Adebayor' 'Wayne Rooney'
 'Sergio Agüero' 'Frank Lampard' 'Franck Ribéry' "Samuel Eto'o"
 'Gianluigi Buffon' 'Cesc Fàbregas' 'Michael Ballack' 'Sergio Ramos'
 'Nemanja Vidić' 'Didier Drogba' 'Edwin van der Sar' 'Ruud van Nistelrooy'
 'Andrés Iniesta' 'Edin Džeko' 'Ryan Giggs' 'Thierry Henry'
 'Luís Fabiano Clemente' 'Diego Forlán' 'Yoann Gourcuff'
 'Júlio César Soares Espíndola' 'Maicon' 'Diego Ribas da Cunha'
 'John Terry' 'Yaya Touré' 'Wesley Sneijder' 'Xabi Alonso' 'Carles Puyol'
 'Mesut Özil' 'Arjen Robben' 'Thomas Müller' 'Bastian Schweinsteiger'
 'Asamoah Gyan' 'Júlio César' 'Miroslav Klose' 'Philipp Lahm' 'Dani Alves'
 'Luis Suárez Díaz' 'Neymar' 'Éric Abidal' 'Karim Benzema' 'Nani'
 'Gerard Piqué' 'Radamel Falcao' 'Andrea Pirlo' 'Robin van Persie'
 'Manuel Neuer' 'Sergio Busquets' 'Mario Balotelli' '

In [20]:
print(p_08_18[8])

Zlatan Ibrahimović


In [21]:
#!pip install --upgrade Selenium
#!pip install --upgrade ChromeDriver

In [22]:
print(p_08_18[8].lower())

zlatan ibrahimović


In [23]:
import unicodedata

lower_list = []

for i in p_08_18:
    #j = i.split()
    #q = "-".join(j)
    j = i.lower()
    
    slug =  unicodedata.normalize('NFD', j).encode('ascii', 'ignore').decode('ascii')
    #print(slug)
    lower_list.append(slug)
    
    
print(lower_list)

['cristiano ronaldo', 'lionel messi', 'fernando torres', 'iker casillas', 'xavi', 'andrei arshavin', 'david villa', 'kaka', 'zlatan ibrahimovic', 'steven gerrard', 'marcos senna', 'emmanuel adebayor', 'wayne rooney', 'sergio aguero', 'frank lampard', 'franck ribery', "samuel eto'o", 'gianluigi buffon', 'cesc fabregas', 'michael ballack', 'sergio ramos', 'nemanja vidic', 'didier drogba', 'edwin van der sar', 'ruud van nistelrooy', 'andres iniesta', 'edin dzeko', 'ryan giggs', 'thierry henry', 'luis fabiano clemente', 'diego forlan', 'yoann gourcuff', 'julio cesar soares espindola', 'maicon', 'diego ribas da cunha', 'john terry', 'yaya toure', 'wesley sneijder', 'xabi alonso', 'carles puyol', 'mesut ozil', 'arjen robben', 'thomas muller', 'bastian schweinsteiger', 'asamoah gyan', 'julio cesar', 'miroslav klose', 'philipp lahm', 'dani alves', 'luis suarez diaz', 'neymar', 'eric abidal', 'karim benzema', 'nani', 'gerard pique', 'radamel falcao', 'andrea pirlo', 'robin van persie', 'manuel 

In [24]:
print(lower_list[8])

zlatan ibrahimovic


In [25]:
#Player name, player_id
#Manually scraped becuase it takes more than 180h+ for jupyter notebook/code and risk of memory error if interent fails 
my_dict = {"cristiano ronaldo": 8198, "lionel messi": 28003, "fernando torres": 7767, "iker casillas": 3979,"xavi": 7607, 
           "andrei arshavin":15378, "david villa": 7980, "kaka": 3366, "zlatan ibrahimovic": 3455, "steven gerrard":3109,
          "marcos senna": 7840, "emmanuel adebayor":8883, "wayne rooney": 3332, "sergio aguero": 26399,
           "frank lampard": 3169,"franck ribery":22068, "samuel eto'o": 4257, "gianluigi buffon": 5023, 
           "cesc fabregas":8806, "michael ballack": 63, "sergio ramos": 25557, "nemanja vidic": 19726, 
           "didier drogba": 3924, "edwin van der sar": 3516, "ruud van nistelrooy": 3407,"andres iniesta": 7600, 
           "edin dzeko":28396, "ryan giggs": 3406, "thierry henry": 3207, "luis fabiano clemente": 19802,
           "diego forlan": 3408, "yoann gourcuff":18912, "julio cesar soares espindola": 22412, "maicon": 18301,
           "diego ribas da cunha": 4248, "john terry": 3160, "yaya toure": 13091,"wesley sneijder": 4673, 
           "xabi alonso":7476, "carles puyol": 7594, "mesut ozil": 35664, "arjen robben":4360, "thomas muller": 58358,
          "bastian schweinsteiger": 2514, "asamoah gyan": 26802,"julio cesar": 22412, "miroslav klose": 10,
           "philipp lahm": 2219, "dani alves": 15951,"luis suarez diaz":44352,"neymar": 68290, "eric abidal": 5283, 
          "karim benzema": 18922, "nani": 33706, "gerard pique": 18944, "radamel falcao": 39152, "andrea pirlo": 5817,
          "robin van persie": 4380,"manuel neuer": 17259, "sergio busquets": 65230, "mario balotelli":45416, 
          "gareth bale": 39381, "robert lewandowski": 38253, "edinson cavani": 48280, "thiago silva":29241, 
          "eden hazard": 50202, "james rodriguez": 88103, "toni kroos": 31909, "angel di maria": 45320, 
           "diego costa": 44779, "mario gotze": 74842, "javier mascherano": 19981,"thibaut courtois": 108390, 
           "paul pogba": 122153, "alexis sanchez":40433, "arturo vidal": 37666, "kevin de bruyne": 88755,
           "ivan rakitic": 32467, "antoine griezmann": 125781, "riyad mahrez": 171424, "jamie vardy":197838,
           "pepe": 14132, "pierre-emerick aubameyang": 58864, "rui patricio":45026, "luka modric": 27992, 
           "dimitri payet":37647, "paulo dybala":206050, "diego godin": 54928, "gonzalo higuain":39153,"koke": 74229,
           "hugo lloris": 17965, "kylian mbappe": 342229, "n'golo kante":225083, "harry kane":132098,"isco":85288,
           "marcelo": 44501, "david de gea": 59377, "leonardo bonucci": 39983, "sadio mane": 200512, "jan oblak": 121483,
           "mats hummels": 39728, "philippe coutinho": 80444, "dries mertens": 56416, "mohamed salah":148455, 
           "raphael varane": 164770, "roberto firmino":131789, "alisson": 105470, "mario mandzukic":34572, 
           "virgil van dijk": 139208, "bernardo silva": 241641,"frenkie de jong":326330, "raheem sterling":134425, 
           "matthijs de ligt": 326031, "trent alexander-arnold": 314353, "dusan tadic": 36139, "son heung-min":91845,
           "kalidou koulibaly": 93128, "marc-andre ter stegen": 74857, "georginio wijnaldum": 49499, "joao felix":462250,
           "marquinhos": 181767, "donny van de beek": 288255, "jorginho": 102017, "gianluigi donnarumma":315858,
           "erling haaland": 418560, "romelu lukaku":96341, "giorgio chiellini": 29260, "luis suarez":44352, 
           "simon kjr":48859, "simon kjaer": 48859, "mason mount": 346483, "bruno fernandes": 240306, 
           "lautaro martinez":406625, "pedri": 683840, "phil foden": 406635, "gerard moreno": 177467, 
           "nicolo barella": 255942, "ruben dias": 258004, "cesar azpilicueta": 57500, "vinicius junior": 371998,
           "sebastien haller":181375, "fabinho": 225693, "rafael leao": 357164, "casemiro": 16306,
           "luis diaz": 480692, "dusan vlahovic": 357498, "joao cancelo": 182712, "joshua kimmich": 161056,
           "mike maignan": 182906, "christopher nkunku": 344381, "darwin nunez": 546543, "antonio rudiger": 86202,
           "rodri": 357565, "julian alvarez": 576024, "victor osimhen": 401923, "yassine bounou":207834, "buno": 207834,
           "ilkay gundogan":53622, "emiliano martinez":111873, "khvicha kvaratskhelia": 502670, "jude bellingham":581678,
           "kim min-jae":503482, "kim min jae":503482, "andre onana": 234509, "bukayo saka":433177, 
           "josko gvardiol":475959, "josko gvardiol":475959, "jamal musiala":580195, "randal kolo muani":487969,
           "martin degaard": 316264, "martin-odegaard":316264}

##Took 1h and 30 min

In [26]:
print(len(list(my_dict.keys())))


170


In [27]:
### All the following strings are formatted with player_id in the actual code 
# Details_stats = "https://www.transfermarkt.com/lionel-messi/torenachminute/spieler/8198/saison//verein/0/liga/0/wettbewerb//pos/0/trainer_id/0/plus/1"
    #Season, Competition Club, 15min intervals of scored goals, Ekstra time, Total amount of goals 
    
    
# Trophies = https://www.transfermarkt.com/mario-mandzukic/erfolge/spieler/34572
    # Kind of trophies, which year/season the specific kind of trophy was won in.


# National team = https://www.transfermarkt.com/mario-mandzukic/nationalmannschaft/spieler/34572/verein_id/3556/hauptwettbewerb//wettbewerb_id//start/2007-11-17/ende/2018-07-15/nurEinsatz/0/plus/1
    #Kind of match in which tournament, Club, Opponant, Date, Result, Goals, Assists, Own goals, 
    # Minute for what kind of card was given to the player, minutes played of the match

National_column_names = ["Group", "Venue", "Date", "Home Team", "Away Team", "Result", "Pos", "Goal", "Assist", "Own Goal", "Yellow",
     "Double Yellow", "Red", "Substitutions on", "Substitutions off", "Minutes Played"]

# Create the pandas DataFrame 
national_messi_df = pd.DataFrame(national_messi, columns = National_column_names)     
    
    
# Stats per tournament = https://www.transfermarkt.com/mario-mandzukic/detaillierteleistungsdaten/spieler/34572
    #Competition, Club name, appearances for the club, Goals, asissists, cards, Minutes Played
    #The aboed are mentioned for each type of legue (National legue, Domestic cups and International cups)

# Stats per club = https://www.transfermarkt.com/mario-mandzukic/leistungsdatenverein/spieler/34572
    #Club name, appearances for the club, Goals, asissists, cards, Minutes Played, 
    #The above are mentioned for each league the player played in 


players = []
for year in range(2008,2024): 
    #print(year)
    m = ballon_2008_2018.loc[ballon_2008_2018["Year"]== year]
    
    for i in m["Player"].values: 
        slug =  unicodedata.normalize('NFD', i).encode('ascii', 'ignore').decode('ascii')
        lowered = slug.lower()
        try: 
            if lowered not in players:
                players.append(lowered)
                player_id = my_dict[lowered]
                print(lowered, player_id)
                Detailed_url = f"https://www.transfermarkt.com/lionel-messi/torenachminute/spieler/{player_id}/saison//verein/0/liga/0/wettbewerb//pos/0/trainer_id/0/plus/1"
                Trophies_url = f"https://www.transfermarkt.com/mario-mandzukic/erfolge/spieler/{player_id}"
                
                National_url = f"https://www.transfermarkt.com/mario-mandzukic/nationalmannschaft/spieler/{player_id}/verein_id/3556/hauptwettbewerb//wettbewerb_id//start/2007-11-17/ende/2018-07-15/nurEinsatz/0/plus/1"
                Stats_per_torunament_url = f"https://www.transfermarkt.com/mario-mandzukic/detaillierteleistungsdaten/spieler/{player_id}"
                Stats_per_club = f"https://www.transfermarkt.com/mario-mandzukic/leistungsdatenverein/spieler/{player_id}"
                
                
                ### The detailed df
                Detailed = get_data_from_div_detailed(Detailed_url)
                df_detailed = pd.DataFrame(Detailed)

                ### The trophies
                trophies_df = get_data_from_div_trophies(Trophies_url)
                df_trophies = pd.DataFrame(trophies_df)
                
                correted_seasons = correct_the_seasons(df_trophies)
                df_trophies[2] = correted_seasons
                #print(df_trophies)
                
                ### National Teams
                national_stats = get_data_from_div_national(National_url)
                national_stats_df = pd.DataFrame(national_stats, columns = National_column_names)  
                
                #national_messi = get_data_from_div_national(National_url)
                
                
                
                #players.append(df_player)
                break
                
        except: 
            print("Player is missing")
       # break
        #result_zlatan = get_data_from_div(zlatan)

        # Convert to DataFrame
        #df_zlatan = pd.DataFrame(result_zlatan)
        

cristiano ronaldo 8198
                                 0  \
0    3x The Best FIFA Men's Player   
1            5x Winner Ballon d'Or   
2    4x UEFA Best Player in Europe   
3       14x Footballer of the Year   
4             1x FIFA Puskás Award   
5              19x Top goal scorer   
6            8x Player of the Year   
7       2x TM-Player of the season   
8             1x European champion   
9       5x Champions League winner   
10   4x FIFA Club World Cup winner   
11             3x English Champion   
12             2x Spanish champion   
13             2x Italian champion   
14         3x Uefa Supercup winner   
15   1x Winner UEFA Nations League   
16        1x English FA Cup winner   
17           2x Spanish cup winner   
18           1x Italian cup winner   
19    2x English League Cup winner   
20     2x Spanish Super Cup winner   
21     2x Italian Super Cup winner   
22     2x English Super Cup winner   
23  1x Portuguese Super Cup winner   

                          

                                           0  \
0                  1x Footballer of the Year   
1                      1x Player of the Year   
2                        1x World Cup winner   
3                       2x European champion   
4                 4x Champions League winner   
5              2x FIFA Club World Cup winner   
6                        8x Spanish champion   
7                    2x Uefa Supercup winner   
8                      3x Spanish cup winner   
9                6x Spanish Super Cup winner   
10            1x Under-20 World Cup champion   
11                        1x Qatari champion   
12  1x Qatari Cup Winner (Emir of Qatar Cup)   

                                   1  \
0                             [2005]   
1                               [07]   
2                             [2010]   
3                       [2012, 2008]   
4                   [14, 10, 08, 05]   
5                       [2012, 2010]   
6   [14, 12, 10, 09, 08, 05, 04, 98]   
7      

                                0       1        2
0       1x Footballer of the Year  [2016]   [2016]
1              1x Top goal scorer    [15]  [15/16]
2           1x Player of the Year    [15]  [15/16]
3             1x World Cup winner  [2018]   [2018]
4         1x Uefa Supercup winner    [18]  [18/19]
5         1x Europa League winner    [17]  [17/18]
6   1x Winner UEFA Nations League  [2021]   [2021]
7           1x Spanish cup winner    [20]  [20/21]
8     1x Spanish Super Cup winner    [14]  [14/15]
9   1x European Under-19 champion  [2010]   [2010]
10   1x Spanish 2nd tier champion    [09]  [09/10]
gianluigi buffon 5023
                                    0  \
0       1x UEFA Best Player in Europe   
1           1x Footballer of the Year   
2          2x TM-Player of the season   
3               1x Player of the Year   
4                 1x World Cup winner   
5                10x Italian champion   
6                  1x French champion   
7                  1x Uefa Cup winner 

                                0                               1  \
0   1x UEFA Best Player in Europe                          [2023]   
1       5x Footballer of the Year  [2023, 2022, 2021, 2020, 2019]   
2              6x Top goal scorer        [22, 22, 22, 20, 20, 18]   
3      2x TM-Player of the season                    [2023, 2020]   
4           1x Player of the Year                            [22]   
5      1x Champions League winner                            [22]   
6             1x English Champion                            [22]   
7         1x Uefa Supercup winner                            [23]   
8        1x English FA Cup winner                          [2023]   
9            1x German cup winner                            [20]   
10           1x Austrian champion                            [18]   
11         1x Austrian cup winner                            [18]   

                                             2  
0                                       [2023]  
1   

In [24]:
for i, j in enumerate(lower_list): 
    f'df_{i}' = j
    print(df_i)

SyntaxError: cannot assign to f-string expression (3758550017.py, line 2)

https://www.transfermarkt.com/zlatan-ibrahimovic/profil/spieler/3450


In [ ]:
print(df_neymar)